In [1]:
from pathlib import Path
import pandas as pd
#import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gzip
import pickle
import json
#import binpickle
#from natural.size import binarysize

In [2]:
topics = pd.read_json('data/trec_2022_wikiprojects.json.gz', lines = True)
topics

,id,title,rel_docs
0,1,1000 Women in Religion,"[29305, 345779, 391183, 478677, 1607259, 18032..."
1,2,A Cappella,"[2411, 71307, 152558, 165101, 304559, 375332, ..."
2,3,A Song of Ice and Fire,"[12300, 12301, 12302, 713577, 713590, 713625, ..."
3,4,AIDS,"[1908, 12747, 14573, 23739, 26214, 26436, 2884..."
4,5,Abandoned Articles,"[41169, 69181, 69748, 180245, 201817, 374129, ..."
...,...,...,...
2909,2910,Zimbabwe/Rhodesia task force,"[17355, 26219, 32927, 99228, 168006, 177496, 1..."
2910,2911,Zoo,"[6198, 6201, 20522, 36901, 37308, 47004, 59922..."
2911,2912,Zoroastrianism,"[1178, 5326, 8539, 10671, 13877, 14653, 19522,..."
2912,2913,liberalism,"[418690, 460286, 482693, 496817, 859617, 99472..."


In [3]:
with open('data/selected_topics_22.txt') as f:
    lines = f.readlines()

In [4]:
selected_topics = []
with open('data/selected_topics_22.txt', encoding='utf8') as f:
    for line in f:
        selected_topics.append(line.strip())

In [5]:
#selected_topics

In [6]:
selected_topics_df = topics[topics['title'].isin(selected_topics)]
selected_topics_df

,id,title,rel_docs
83,84,Agriculture,"[572, 627, 678, 903, 1193, 1542, 1634, 3751, 3..."
110,111,Amphibians and Reptiles,"[621, 809, 1380, 6641, 8311, 8937, 13134, 1446..."
186,187,Architecture,"[682, 954, 1170, 1315, 1322, 1324, 1325, 1435,..."
264,265,Astronomy,"[39, 308, 580, 664, 736, 748, 791, 798, 799, 1..."
269,270,Athletics,"[5729, 8490, 9623, 10391, 12231, 13791, 16078,..."
...,...,...,...
2740,2741,Video games,"[785, 1203, 1980, 2078, 2116, 2141, 2197, 2215..."
2757,2758,Visual arts,"[593, 752, 864, 1212, 1578, 1642, 1647, 1881, ..."
2858,2859,Women,"[890, 974, 984, 2126, 2519, 2529, 2847, 3013, ..."
2866,2867,Women writers,"[339, 984, 1000, 1002, 2029, 2314, 2834, 2847,..."


In [58]:
len(selected_topics_df['rel_docs'][83])

7421

In [71]:
relevant_docs = []
for row in selected_topics_df['rel_docs']:
    for i in row:
        relevant_docs.append(i)

In [75]:
relevant_docs = list(set(relevant_docs))

In [82]:
#with gzip.open('../trec_2022_articles_discrete.json.gz', 'rt') as fin:
data = []
file = 'data/trec_2022_articles_discrete.json.gz'
#for file in glob.glob('logs/*.json'):
with open(file, 'rb') as fd:
    f = gzip.GzipFile(fileobj=fd)
    for line in f:
        #if json.loads(line)['page_id'] in relevant_docs and json.loads(line)['pred_qual'] > 0.5:
        doc = json.loads(line)
        lst = [doc['page_id'], doc['pred_qual']]
        data.append(lst)

meta = pd.DataFrame(data, columns = ['page_id', 'pred_qual'])

In [84]:
meta = pd.DataFrame(data, columns = ['page_id', 'pred_qual'])
meta

,page_id,pred_qual
0,12,0.723514
1,25,0.849966
2,39,0.888689
3,290,0.846403
4,303,0.922363
...,...,...
6460233,70194419,0.359100
6460234,70194480,0.283824
6460235,70194481,0.662496
6460236,70194489,0.371109


In [85]:
def seed_rel_docs(row, meta):
    df = meta[meta['page_id'].isin(row)]
    df = df.sort_values(by='pred_qual', ascending = False)
    #print(df.head(5))
    return df['page_id'].values

In [86]:
selected_topics_df['sorted_articles'] = selected_topics_df['rel_docs'].apply(lambda x: seed_rel_docs(x, meta))

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [87]:
selected_topics_df

,id,title,rel_docs,sorted_articles,seed_articles
83,84,Agriculture,"[572, 627, 678, 903, 1193, 1542, 1634, 3751, 3...","[585460, 20656228, 11107, 333606, 62696, 18214...","[585460, 20656228, 11107, 333606, 62696, 18214..."
110,111,Amphibians and Reptiles,"[621, 809, 1380, 6641, 8311, 8937, 13134, 1446...","[26364308, 515343, 34248635, 484824, 34240006,...","[21446213, 41483998, 16052266]"
186,187,Architecture,"[682, 954, 1170, 1315, 1322, 1324, 1325, 1435,...","[18934361, 325018, 69841, 65456, 308726, 54044...","[53212069, 5219405, 3555585, 2572563, 20459568..."
264,265,Astronomy,"[39, 308, 580, 664, 736, 748, 791, 798, 799, 1...","[15736, 621989, 344783, 1102000, 16459, 203433...",[11149908]
269,270,Athletics,"[5729, 8490, 9623, 10391, 12231, 13791, 16078,...","[2935379, 143673, 1321034, 6433331, 2593778, 9...",[8341524]
...,...,...,...,...,...
2740,2741,Video games,"[785, 1203, 1980, 2078, 2116, 2141, 2197, 2215...","[208074, 38062234, 15526, 302111, 24673, 59351...",[40648600]
2757,2758,Visual arts,"[593, 752, 864, 1212, 1578, 1642, 1647, 1881, ...","[148118, 325018, 1586702, 149045, 26094, 64689...","[51304026, 920902, 64931391, 7802416, 47169352..."
2858,2859,Women,"[890, 974, 984, 2126, 2519, 2529, 2847, 3013, ...","[250340, 19047, 156223, 663405, 28857837, 1550...","[28098592, 63405924, 67373999, 56898595, 66250..."
2866,2867,Women writers,"[339, 984, 1000, 1002, 2029, 2314, 2834, 2847,...","[197446, 140973, 63689, 31868207, 295860, 1628...","[41711402, 55049149, 368106, 20786388, 4864121..."


In [88]:
selected_topics_df['seed_articles'] = selected_topics_df['sorted_articles'].apply(lambda x: x[:10])
selected_topics_df

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,id,title,rel_docs,sorted_articles,seed_articles
83,84,Agriculture,"[572, 627, 678, 903, 1193, 1542, 1634, 3751, 3...","[585460, 20656228, 11107, 333606, 62696, 18214...","[585460, 20656228, 11107, 333606, 62696, 18214..."
110,111,Amphibians and Reptiles,"[621, 809, 1380, 6641, 8311, 8937, 13134, 1446...","[26364308, 515343, 34248635, 484824, 34240006,...","[26364308, 515343, 34248635, 484824, 34240006,..."
186,187,Architecture,"[682, 954, 1170, 1315, 1322, 1324, 1325, 1435,...","[18934361, 325018, 69841, 65456, 308726, 54044...","[18934361, 325018, 69841, 65456, 308726, 54044..."
264,265,Astronomy,"[39, 308, 580, 664, 736, 748, 791, 798, 799, 1...","[15736, 621989, 344783, 1102000, 16459, 203433...","[15736, 621989, 344783, 1102000, 16459, 203433..."
269,270,Athletics,"[5729, 8490, 9623, 10391, 12231, 13791, 16078,...","[2935379, 143673, 1321034, 6433331, 2593778, 9...","[2935379, 143673, 1321034, 6433331, 2593778, 9..."
...,...,...,...,...,...
2740,2741,Video games,"[785, 1203, 1980, 2078, 2116, 2141, 2197, 2215...","[208074, 38062234, 15526, 302111, 24673, 59351...","[208074, 38062234, 15526, 302111, 24673, 59351..."
2757,2758,Visual arts,"[593, 752, 864, 1212, 1578, 1642, 1647, 1881, ...","[148118, 325018, 1586702, 149045, 26094, 64689...","[148118, 325018, 1586702, 149045, 26094, 64689..."
2858,2859,Women,"[890, 974, 984, 2126, 2519, 2529, 2847, 3013, ...","[250340, 19047, 156223, 663405, 28857837, 1550...","[250340, 19047, 156223, 663405, 28857837, 1550..."
2866,2867,Women writers,"[339, 984, 1000, 1002, 2029, 2314, 2834, 2847,...","[197446, 140973, 63689, 31868207, 295860, 1628...","[197446, 140973, 63689, 31868207, 295860, 1628..."


In [95]:
topic_info = selected_topics_df[['id', 'title', 'seed_articles']]

In [96]:
topic_info = topic_info.explode('seed_articles', ignore_index = True)
topic_info.rename(columns={'seed_articles':'page_id'},inplace=True)

In [97]:
topic_info.dropna(inplace=True)

In [98]:
topic_info

,id,title,page_id
0,84,Agriculture,585460
1,84,Agriculture,20656228
2,84,Agriculture,11107
3,84,Agriculture,333606
4,84,Agriculture,62696
...,...,...,...
995,2872,Women's sport,9509330
996,2872,Women's sport,31804969
997,2872,Women's sport,3517532
998,2872,Women's sport,21031813


In [100]:
topic_info.to_csv('data/topic_info_2022.csv')

In [102]:
seed_articles = set(topic_info['page_id'].values)
len(seed_articles)

773

In [130]:
data = []
file = 'data/trec_corpus_20220301_text.json.gz'
#for file in glob.glob('logs/*.json'):
with open(file, 'rb') as fd:
    f = gzip.GzipFile(fileobj=fd)
    for line in f:
        parse = json.loads(line)
        if parse['id'] in seed_articles:
            data.append(parse)# json.loads(line)['id'] in relevant_docs:
            

pages = pd.DataFrame(data)

In [ ]:
data = []
file = 'data/trec_corpus_20220301_plain.json.gz'
#for file in glob.glob('logs/*.json'):
with open(file, 'rb') as fd:
    f = gzip.GzipFile(fileobj=fd)
    for line in f:
        parse = json.loads(line)
        if parse['id'] in seed_articles:
            data.append(parse)# json.loads(line)['id'] in relevant_docs:
            

plain_pages = pd.DataFrame(data)

In [2]:
#pages['text'][0]

In [123]:
plain_pages.to_csv('data/selected_pages_2022_corpus.csv', index = False)

In [2]:
plain_pages = pd.read_csv('data/selected_pages_2022_corpus.csv')
plain_pages

,id,title,url,plain
0,16897736,Jon Batiste,https://en.wikipedia.org/wiki/Jon_Batiste,"Jonathan Michael Batiste (born November 11, 19..."
1,2853,Aberdeen Bestiary,https://en.wikipedia.org/wiki/Aberdeen_Bestiary,The Aberdeen Bestiary (Aberdeen University Lib...
2,882558,Ray Allen,https://en.wikipedia.org/wiki/Ray_Allen,"Walter Ray Allen Jr. (born July 20, 1975) is a..."
3,13467,Hamburg,https://en.wikipedia.org/wiki/Hamburg,"Hamburg (, ; ), officially the Free and Hanse..."
4,853925,Randy Kraft,https://en.wikipedia.org/wiki/Randy_Kraft,"Randy Steven Kraft (born March 19, 1945) is an..."
...,...,...,...,...
768,17741,Ludwig Wittgenstein,https://en.wikipedia.org/wiki/Ludwig_Wittgenstein,Ludwig Josef Johann Wittgenstein ( ; ; 26 Apri...
769,2575928,Adelaide Hall,https://en.wikipedia.org/wiki/Adelaide_Hall,Adelaide Louise Hall (20 October 1901 – 7 Nove...
770,621989,Viktor Ambartsumian,https://en.wikipedia.org/wiki/Viktor_Ambartsumian,"Viktor Amazaspovich Ambartsumian (; , Viktor H..."
771,63299,Václav Havel,https://en.wikipedia.org/wiki/V%C3%A1clav_Havel,Václav Havel (; 5 October 193618 December 2011...


In [3]:
#pages = pd.read_json('data/trec_corpus.json.gz', lines = True, nrows=100)
#pages = pages.drop_duplicates('page_id')
#pages

In [4]:
MEDIA_PREFIXES = ['File', 'Image', 'Media']
CAT_PREFIXES = ['Category']

MEDIA_ALIASES = {
    "ab": ["Медиа", "Файл", "Афаил", "Амедиа", "Изображение"],
    "ace": ["Beureukaih", "Gambar", "Alat", "Berkas"],
    "ady": ["Медиа"],
    "af": ["Lêer", "Beeld"],
    "als": ["Medium", "Datei", "Bild"],
    "am": ["ፋይል", "ስዕል"],
    "an": ["Imachen", "Imagen"],
    "ang": ["Ymele", "Biliþ"],
    "ar": ["ميديا", "صورة", "وسائط", "ملف"],
    "arc": ["ܠܦܦܐ", "ܡܝܕܝܐ"],
    "arz": ["ميديا", "صورة", "وسائط", "ملف"],
    "as": ["চিত্ৰ", "चित्र", "চিত্র", "মাধ্যম"],
    "ast": ["Imaxen", "Ficheru", "Imaxe", "Archivu", "Imagen", "Medios"],
    "atj": ["Tipatcimoctakewin", "Natisinahikaniwoc"],
    "av": ["Медиа", "Файл", "Изображение"],
    "ay": ["Medio", "Archivo", "Imagen"],
    "az": ["Mediya", "Şəkil", "Fayl"],
    "azb": ["رسانه", "تصویر", "مدیا", "فایل", "رسانه‌ای"],
    "ba": ["Медиа", "Рәсем", "Файл", "Изображение"],
    "bar": ["Medium", "Datei", "Bild"],
    "bat-smg": ["Vaizdas", "Medėjė", "Abruozdielis"],
    "bcl": ["Medio", "Ladawan"],
    "be": ["Мультымедыя", "Файл", "Выява"],
    "be-x-old": ["Мэдыя", "Файл", "Выява"],
    "bg": ["Медия", "Файл", "Картинка"],
    "bh": ["मीडिया", "चित्र"],
    "bjn": ["Barakas", "Gambar", "Berkas"],
    "bm": ["Média", "Fichier"],
    "bn": ["চিত্র", "মিডিয়া"],
    "bpy": ["ছবি", "মিডিয়া"],
    "br": ["Skeudenn", "Restr"],
    "bs": ["Mediji", "Slika", "Datoteka", "Medija"],
    "bug": ["Gambar", "Berkas"],
    "bxr": ["Файл", "Меди", "Изображение"],
    "ca": ["Fitxer", "Imatge"],
    "cbk-zam": ["Medio", "Archivo", "Imagen"],
    "cdo": ["文件", "媒體", "圖像", "檔案"],
    "ce": ["Хlум", "Медиа", "Сурт", "Файл", "Медйа", "Изображение"],
    "ceb": ["Payl", "Medya", "Imahen"],
    "ch": ["Litratu"],
    "ckb": ["میدیا", "پەڕگە"],
    "co": ["Immagine"],
    "crh": ["Медиа", "Resim", "Файл", "Fayl", "Ресим"],
    "cs": ["Soubor", "Média", "Obrázok"],
    "csb": ["Òbrôzk", "Grafika"],
    "cu": ["Видъ", "Ви́дъ", "Дѣло", "Срѣдьства"],
    "cv": ["Медиа", "Ӳкерчĕк", "Изображение"],
    "cy": ["Delwedd"],
    "da": ["Billede", "Fil"],
    "de": ["Medium", "Datei", "Bild"],
    "din": ["Ciɛl", "Apamduööt"],
    "diq": ["Medya", "Dosya"],
    "dsb": ["Wobraz", "Dataja", "Bild", "Medija"],
    "dty": ["चित्र", "मिडिया"],
    "dv": ["ފައިލު", "މީޑިއާ", "ފައިލް"],
    "el": ["Εικόνα", "Αρχείο", "Μέσο", "Μέσον"],
    "eml": ["Immagine"],
    "eo": ["Dosiero", "Aŭdvidaĵo"],
    "es": ["Medio", "Archivo", "Imagen"],
    "et": ["Pilt", "Fail", "Meedia"],
    "eu": ["Irudi", "Fitxategi"],
    "ext": ["Archivu", "Imagen", "Mediu"],
    "fa": ["رسانه", "تصویر", "مدیا", "پرونده", "رسانه‌ای"],
    "ff": ["Média", "Fichier"],
    "fi": ["Kuva", "Tiedosto"],
    "fiu-vro": ["Pilt", "Meediä"],
    "fo": ["Miðil", "Mynd"],
    "fr": ["Média", "Fichier"],
    "frp": ["Émâge", "Fichiér", "Mèdia"],
    "frr": ["Medium", "Datei", "Bild"],
    "fur": ["Immagine", "Figure"],
    "fy": ["Ofbyld"],
    "ga": ["Íomhá", "Meán"],
    "gag": ["Mediya", "Medya", "Resim", "Dosya", "Dosye"],
    "gan": ["媒体文件", "文件", "文檔", "档案", "媒體", "图像", "圖像", "媒体", "檔案"],
    "gd": ["Faidhle", "Meadhan"],
    "gl": ["Imaxe", "Ficheiro", "Arquivo", "Imagem"],
    "glk": ["رسانه", "تصویر", "پرونده", "فاىل", "رسانه‌ای", "مديا"],
    "gn": ["Medio", "Imagen", "Ta'ãnga"],
    "gom": ["माध्यम", "मिडिया", "फायल"],
    "gor": ["Gambar", "Berkas"],
    "got": ["𐍆𐌴𐌹𐌻𐌰"],
    "gu": ["દ્રશ્ય-શ્રાવ્ય (મિડિયા)", "દ્રશ્ય-શ્રાવ્ય_(મિડિયા)", "ચિત્ર"],
    "gv": ["Coadan", "Meanyn"],
    "hak": ["文件", "媒體", "圖像", "檔案"],
    "haw": ["Kiʻi", "Waihona", "Pāpaho"],
    "he": ["תמונה", "קו", "מדיה", "קובץ"],
    "hi": ["मीडिया", "चित्र"],
    "hif": ["file", "saadhan"],
    "hr": ["Mediji", "DT", "Slika", "F", "Datoteka"],
    "hsb": ["Wobraz", "Dataja", "Bild"],
    "ht": ["Imaj", "Fichye", "Medya"],
    "hu": ["Kép", "Fájl", "Média"],
    "hy": ["Պատկեր", "Մեդիա"],
    "ia": ["Imagine", "Multimedia"],
    "id": ["Gambar", "Berkas"],
    "ig": ["Nká", "Midia", "Usòrò", "Ákwúkwó orünotu", "Ákwúkwó_orünotu"],
    "ii": ["媒体文件", "文件", "档案", "图像", "媒体"],
    "ilo": ["Midia", "Papeles"],
    "inh": ["Медиа", "Файл", "Изображение"],
    "io": ["Imajo", "Arkivo"],
    "is": ["Miðill", "Mynd"],
    "it": ["Immagine"],
    "ja": ["メディア", "ファイル", "画像"],
    "jbo": ["velsku", "datnyvei"],
    "jv": ["Barkas", "Medhia", "Gambar", "Médhia"],
    "ka": ["მედია", "სურათი", "ფაილი"],
    "kaa": ["Swret", "Таспа", "سۋرەت", "Taspa", "Su'wret", "Сурет", "تاسپا"],
    "kab": ["Tugna"],
    "kbd": ["Медиа", "Файл"],
    "kbp": ["Média", "Fichier"],
    "kg": ["Fisye"],
    "kk": ["Swret", "سۋرەت", "Таспа", "Taspa", "Сурет", "تاسپا"],
    "kl": ["Billede", "Fiileq", "Fil"],
    "km": ["ឯកសារ", "រូបភាព", "មេឌា", "មីឌា"],
    "kn": ["ಚಿತ್ರ", "ಮೀಡಿಯ"],
    "ko": ["미디어", "파일", "그림"],
    "koi": ["Медиа", "Файл", "Изображение"],
    "krc": ["Медиа", "Файл", "Изображение"],
    "ks": ["میڈیا", "فَیِل"],
    "ksh": ["Beld", "Meedije", "Medie", "Belld", "Medium", "Datei", "Meedijum", "Bild"],
    "ku": ["میدیا", "پەڕگە", "Medya", "Wêne"],
    "kv": ["Медиа", "Файл", "Изображение"],
    "kw": ["Restren"],
    "ky": ["Медиа", "Файл"],
    "la": ["Imago", "Fasciculus"],
    "lad": ["Dossia", "Medya", "Archivo", "Dosya", "Imagen", "Meddia"],
    "lb": ["Fichier", "Bild"],
    "lbe": ["Медиа", "Сурат", "Изображение"],
    "lez": ["Медиа", "Mediya", "Файл", "Şəkil", "Изображение"],
    "lfn": ["Fix"],
    "li": ["Afbeelding", "Plaetje", "Aafbeilding"],
    "lij": ["Immaggine", "Immagine"],
    "lmo": ["Immagine", "Imàjine", "Archivi"],
    "ln": ["Média", "Fichier"],
    "lo": ["ສື່ອ", "ສື່", "ຮູບ"],
    "lrc": ["رسانه", "تصویر", "رسانه‌ای", "جانیا", "أسگ", "ڤارئسگأر"],
    "lt": ["Vaizdas", "Medija"],
    "ltg": ["Medeja", "Fails"],
    "lv": ["Attēls"],
    "mai": ["मेडिया", "फाइल"],
    "map-bms": ["Barkas", "Medhia", "Gambar", "Médhia"],
    "mdf": ["Медиа", "Няйф", "Изображение"],
    "mg": ["Rakitra", "Sary", "Média"],
    "mhr": ["Медиа", "Файл", "Изображение"],
    "min": ["Gambar", "Berkas"],
    "mk": ["Податотека", "Медија", "Медиум", "Слика"],
    "ml": ["പ്രമാണം", "ചി", "മീഡിയ", "പ്ര", "ചിത്രം"],
    "mn": ["Медиа", "Файл", "Зураг"],
    "mr": ["चित्र", "मिडिया"],
    "mrj": ["Медиа", "Файл", "Изображение"],
    "ms": ["Fail", "Imej"],
    "mt": ["Midja", "Medja", "Stampa"],
    "mwl": ["Multimédia", "Fexeiro", "Ficheiro", "Arquivo", "Imagem"],
    "my": ["ဖိုင်", "မီဒီယာ"],
    "myv": ["Медия", "Артовкс", "Изображение"],
    "mzn": ["رسانه", "تصویر", "مه‌دیا", "مدیا", "پرونده", "رسانه‌ای"],
    "nah": ["Mēdiatl", "Īxiptli", "Imagen"],
    "nap": ["Fiùra", "Immagine"],
    "nds": ["Datei", "Bild"],
    "nds-nl": ["Ofbeelding", "Afbeelding", "Bestaand"],
    "ne": ["मीडिया", "चित्र"],
    "new": ["किपा", "माध्यम"],
    "nl": ["Bestand", "Afbeelding"],
    "nn": ["Fil", "Bilde", "Filpeikar"],
    "no": ["Fil", "Medium", "Bilde"],
    "nov": [],
    "nrm": ["Média", "Fichier"],
    "nso": ["Seswantšho"],
    "nv": ["Eʼelyaaígíí"],
    "oc": ["Imatge", "Fichièr", "Mèdia"],
    "olo": ["Kuva", "Medii", "Failu"],
    "or": ["ମାଧ୍ୟମ", "ଫାଇଲ"],
    "os": ["Ныв", "Медиа", "Файл", "Изображение"],
    "pa": ["ਤਸਵੀਰ", "ਮੀਡੀਆ"],
    "pcd": ["Média", "Fichier"],
    "pdc": ["Medium", "Datei", "Bild", "Feil"],
    "pfl": ["Dadai", "Medium", "Datei", "Bild"],
    "pi": ["मीडिया", "पटिमा"],
    "pl": ["Plik", "Grafika"],
    "pms": ["Figura", "Immagine"],
    "pnb": ["میڈیا", "تصویر", "فائل"],
    "pnt": ["Εικόνα", "Αρχείον", "Εικόναν", "Μέσον"],
    "ps": ["انځور", "رسنۍ", "دوتنه"],
    "pt": ["Multimédia", "Ficheiro", "Arquivo", "Imagem"],
    "qu": ["Midya", "Imagen", "Rikcha"],
    "rm": ["Multimedia", "Datoteca"],
    "rmy": ["Fişier", "Mediya", "Chitro", "Imagine"],
    "ro": ["Fişier", "Imagine", "Fișier"],
    "roa-rup": ["Fişier", "Imagine", "Fișier"],
    "roa-tara": ["Immagine"],
    "ru": ["Медиа", "Файл", "Изображение"],
    "rue": ["Медіа", "Медиа", "Файл", "Изображение", "Зображення"],
    "rw": ["Dosiye", "Itangazamakuru"],
    "sa": ["चित्रम्", "माध्यमम्", "सञ्चिका", "माध्यम", "चित्रं"],
    "sah": ["Миэдьийэ", "Ойуу", "Билэ", "Изображение"],
    "sat": ["ᱨᱮᱫ", "ᱢᱤᱰᱤᱭᱟ"],
    "sc": ["Immàgini"],
    "scn": ["Immagine", "Mmàggini", "Mèdia"],
    "sd": ["عڪس", "ذريعات", "فائل"],
    "se": ["Fiila"],
    "sg": ["Média", "Fichier"],
    "sh": ["Mediji", "Slika", "Медија", "Datoteka", "Medija", "Слика"],
    "si": ["රූපය", "මාධ්‍යය", "ගොනුව"],
    "sk": ["Súbor", "Obrázok", "Médiá"],
    "sl": ["Slika", "Datoteka"],
    "sq": ["Figura", "Skeda"],
    "sr": ["Датотека", "Medij", "Slika", "Медија", "Datoteka", "Медиј", "Medija", "Слика"],
    "srn": ["Afbeelding", "Gefre"],
    "stq": ["Bielde", "Bild"],
    "su": ["Média", "Gambar"],
    "sv": ["Fil", "Bild"],
    "sw": ["Faili", "Picha"],
    "szl": ["Plik", "Grafika"],
    "ta": ["படிமம்", "ஊடகம்"],
    "tcy": ["ಮಾದ್ಯಮೊ", "ಫೈಲ್"],
    "te": ["ఫైలు", "దస్త్రం", "బొమ్మ", "మీడియా"],
    "tet": ["Imajen", "Arquivo", "Imagem"],
    "tg": ["Акс", "Медиа"],
    "th": ["ไฟล์", "สื่อ", "ภาพ"],
    "ti": ["ፋይል", "ሜድያ"],
    "tk": ["Faýl"],
    "tl": ["Midya", "Talaksan"],
    "tpi": ["Fail"],
    "tr": ["Medya", "Resim", "Dosya", "Ortam"],
    "tt": ["Медиа", "Рәсем", "Файл", "Räsem", "Изображение"],
    "ty": ["Média", "Fichier"],
    "tyv": ["Медиа", "Файл", "Изображение"],
    "udm": ["Медиа", "Файл", "Суред", "Изображение"],
    "ug": ["ۋاسىتە", "ھۆججەت"],
    "uk": ["Медіа", "Медиа", "Файл", "Изображение", "Зображення"],
    "ur": ["میڈیا", "تصویر", "وسیط", "زریعہ", "فائل", "ملف"],
    "uz": ["Mediya", "Tasvir", "Fayl"],
    "vec": ["Immagine", "Imàjine", "Mèdia"],
    "vep": ["Pilt", "Fail"],
    "vi": ["Phương_tiện", "Tập_tin", "Hình", "Tập tin", "Phương tiện"],
    "vls": ["Afbeelding", "Ofbeeldienge"],
    "vo": ["Ragiv", "Magod", "Nünamakanäd"],
    "wa": ["Imådje"],
    "war": ["Medya", "Fayl", "Paypay"],
    "wo": ["Xibaarukaay", "Dencukaay"],
    "wuu": ["文件", "档案", "图像", "媒体"],
    "xal": ["Аһар", "Боомг", "Изображение", "Зург"],
    "xmf": ["მედია", "სურათი", "ფაილი"],
    "yi": ["מעדיע", "תמונה", "טעקע", "בילד"],
    "yo": ["Fáìlì", "Amóhùnmáwòrán", "Àwòrán"],
    "za": ["媒体文件", "文件", "档案", "图像", "媒体"],
    "zea": ["Afbeelding", "Plaetje"],
    "zh": ["媒体文件", "F", "文件", "媒體", "档案", "图像", "圖像", "媒体", "檔案"],
    "zh-classical": ["文件", "媒體", "圖像", "檔案"],
    "zh-min-nan": ["tóng-àn", "文件", "媒體", "Mûi-thé", "圖像", "檔案"],
    "zh-yue": ["檔", "档", "文件", "图", "媒體", "圖", "档案", "图像", "圖像", "媒体", "檔案"],
}

CAT_ALIASES = {
    "ab": ["Категория", "Акатегориа"],
    "ace": ["Kawan", "Kategori"],
    "af": ["Kategorie"],
    "ak": ["Nkyekyem"],
    "als": ["Kategorie"],
    "am": ["መደብ"],
    "an": ["Categoría"],
    "ang": ["Flocc"],
    "ar": ["تصنيف"],
    "arc": ["ܣܕܪܐ"],
    "arz": ["تصنيف"],
    "as": ["CAT", "শ্ৰেণী", "श्रेणी", "শ্রেণী"],
    "ast": ["Categoría"],
    "atj": ["Tipanictawin"],
    "av": ["Категория"],
    "ay": ["Categoría"],
    "az": ["Kateqoriya"],
    "azb": ["بؤلمه"],
    "ba": ["Төркөм", "Категория"],
    "bar": ["Kategorie"],
    "bat-smg": ["Kategorija", "Kateguorėjė"],
    "bcl": ["Kategorya"],
    "be": ["Катэгорыя"],
    "be-x-old": ["Катэгорыя"],
    "bg": ["Категория"],
    "bh": ["श्रेणी"],
    "bjn": ["Tumbung", "Kategori"],
    "bm": ["Catégorie"],
    "bn": ["বিষয়শ্রেণী", "വിഭാഗം"],
    "bpy": ["থাক"],
    "br": ["Rummad"],
    "bs": ["Kategorija"],
    "bug": ["Kategori"],
    "bxr": ["Категори", "Категория"],
    "ca": ["Categoria"],
    "cbk-zam": ["Categoría"],
    "cdo": ["分類"],
    "ce": ["Категори", "Тоба", "Кадегар"],
    "ceb": ["Kategoriya"],
    "ch": ["Katigoria"],
    "ckb": ["پ", "پۆل"],
    "co": ["Categoria"],
    "crh": ["Категория", "Kategoriya"],
    "cs": ["Kategorie"],
    "csb": ["Kategòrëjô"],
    "cu": ["Катигорї", "Категория", "Катигорїꙗ"],
    "cv": ["Категори"],
    "cy": ["Categori"],
    "da": ["Kategori"],
    "de": ["Kategorie"],
    "din": ["Bekätakthook"],
    "diq": ["Kategoriye", "Kategori"],
    "dsb": ["Kategorija"],
    "dty": ["श्रेणी"],
    "dv": ["ޤިސްމު"],
    "el": ["Κατηγορία"],
    "eml": ["Categoria"],
    "eo": ["Kategorio"],
    "es": ["CAT", "Categoría"],
    "et": ["Kategooria"],
    "eu": ["Kategoria"],
    "ext": ["Categoría", "Categoria"],
    "fa": ["رده"],
    "ff": ["Catégorie"],
    "fi": ["Luokka"],
    "fiu-vro": ["Katõgooria"],
    "fo": ["Bólkur"],
    "fr": ["Catégorie"],
    "frp": ["Catègorie"],
    "frr": ["Kategorie"],
    "fur": ["Categorie"],
    "fy": ["Kategory"],
    "ga": ["Rang", "Catagóir"],
    "gag": ["Kategori", "Kategoriya"],
    "gan": ["分類", "分类"],
    "gd": ["Roinn-seòrsa"],
    "gl": ["Categoría"],
    "glk": ["جرگه", "رده"],
    "gn": ["Ñemohenda"],
    "gom": ["वर्ग", "श्रेणी"],
    "gor": ["Dalala"],
    "got": ["𐌷𐌰𐌽𐍃𐌰"],
    "gu": ["શ્રેણી", "CAT", "શ્રે"],
    "gv": ["Ronney"],
    "hak": ["分類"],
    "haw": ["Māhele"],
    "he": ["קטגוריה", "קט"],
    "hi": ["श्र", "श्रेणी"],
    "hif": ["vibhag"],
    "hr": ["CT", "KT", "Kategorija"],
    "hsb": ["Kategorija"],
    "ht": ["Kategori"],
    "hu": ["Kategória"],
    "hy": ["Կատեգորիա"],
    "ia": ["Categoria"],
    "id": ["Kategori"],
    "ie": ["Categorie"],
    "ig": ["Ébéonọr", "Òtù"],
    "ii": ["分类"],
    "ilo": ["Kategoria"],
    "inh": ["ОагӀат"],
    "io": ["Kategorio"],
    "is": ["Flokkur"],
    "it": ["CAT", "Categoria"],
    "ja": ["カテゴリ"],
    "jbo": ["klesi"],
    "jv": ["Kategori"],
    "ka": ["კატეგორია"],
    "kaa": ["Sanat", "Kategoriya", "Санат", "سانات"],
    "kab": ["Taggayt"],
    "kbd": ["Категория", "Категориэ"],
    "kbp": ["Catégorie"],
    "kg": ["Kalasi"],
    "kk": ["Sanat", "Санат", "سانات"],
    "kl": ["Sumut_atassuseq", "Kategori", "Sumut atassuseq"],
    "km": ["ចំនាត់ថ្នាក់ក្រុម", "ចំណាត់ក្រុម", "ចំណាត់ថ្នាក់ក្រុម"],
    "kn": ["ವರ್ಗ"],
    "ko": ["분류"],
    "koi": ["Категория"],
    "krc": ["Категория"],
    "ks": ["زٲژ"],
    "ksh": ["Saachjropp", "Saachjrop", "Katejori", "Kategorie", "Saachjrupp", "Kattejori", "Sachjrop"],
    "ku": ["Kategorî", "پۆل"],
    "kv": ["Категория"],
    "kw": ["Class", "Klass"],
    "ky": ["Категория"],
    "la": ["Categoria"],
    "lad": ["Kateggoría", "Katēggoría", "Categoría"],
    "lb": ["Kategorie"],
    "lbe": ["Категория"],
    "lez": ["Категория"],
    "lfn": ["Categoria"],
    "li": ["Categorie", "Kategorie"],
    "lij": ["Categorîa", "Categoria"],
    "lmo": ["Categuria", "Categoria"],
    "ln": ["Catégorie"],
    "lo": ["ໝວດ"],
    "lrc": ["دأسە"],
    "lt": ["Kategorija"],
    "ltg": ["Kategoreja"],
    "lv": ["Kategorija"],
    "mai": ["CA", "श्रेणी"],
    "map-bms": ["Kategori"],
    "mdf": ["Категорие", "Категория"],
    "mg": ["Sokajy", "Catégorie"],
    "mhr": ["Категория", "Категорий"],
    "min": ["Kategori"],
    "mk": ["Категорија"],
    "ml": ["വിഭാഗം", "വി", "വർഗ്ഗം", "വ"],
    "mn": ["Ангилал"],
    "mr": ["वर्ग"],
    "mrj": ["Категори", "Категория"],
    "ms": ["Kategori"],
    "mt": ["Kategorija"],
    "mwl": ["Catadorie", "Categoria"],
    "my": ["ကဏ္ဍ"],
    "myv": ["Категория"],
    "mzn": ["رج", "رده"],
    "nah": ["Neneuhcāyōtl", "Categoría"],
    "nap": ["Categurìa", "Categoria"],
    "nds": ["Kategorie"],
    "nds-nl": ["Categorie", "Kattegerie", "Kategorie"],
    "ne": ["श्रेणी"],
    "new": ["पुचः"],
    "nl": ["Categorie"],
    "nn": ["Kategori"],
    "no": ["Kategori"],
    "nrm": ["Catégorie"],
    "nso": ["Setensele"],
    "nv": ["Tʼááłáhági_átʼéego", "Tʼááłáhági átʼéego"],
    "oc": ["Categoria"],
    "olo": ["Kategourii"],
    "or": ["ବିଭାଗ", "ଶ୍ରେଣୀ"],
    "os": ["Категори"],
    "pa": ["ਸ਼੍ਰੇਣੀ"],
    "pcd": ["Catégorie"],
    "pdc": ["Abdeeling", "Kategorie"],
    "pfl": ["Kadegorie", "Sachgrubb", "Kategorie"],
    "pi": ["विभाग"],
    "pl": ["Kategoria"],
    "pms": ["Categorìa"],
    "pnb": ["گٹھ"],
    "pnt": ["Κατηγορίαν"],
    "ps": ["وېشنيزه"],
    "pt": ["Categoria"],
    "qu": ["Katiguriya"],
    "rm": ["Categoria"],
    "rmy": ["Shopni"],
    "ro": ["Categorie"],
    "roa-rup": ["Categorie"],
    "roa-tara": ["Categoria"],
    "ru": ["Категория", "К"],
    "rue": ["Категория", "Катеґорія"],
    "rw": ["Ikiciro"],
    "sa": ["वर्गः"],
    "sah": ["Категория"],
    "sat": ["ᱛᱷᱚᱠ"],
    "sc": ["Categoria"],
    "scn": ["Catigurìa"],
    "sd": ["زمرو"],
    "se": ["Kategoriija"],
    "sg": ["Catégorie"],
    "sh": ["Kategorija", "Категорија"],
    "si": ["ප්‍රවර්ගය"],
    "sk": ["Kategória"],
    "sl": ["Kategorija"],
    "sq": ["Kategoria", "Kategori"],
    "sr": ["Kategorija", "Категорија"],
    "srn": ["Categorie", "Guru"],
    "stq": ["Kategorie"],
    "su": ["Kategori"],
    "sv": ["Kategori"],
    "sw": ["Jamii"],
    "szl": ["Kategoryjo", "Kategoria"],
    "ta": ["பகுப்பு"],
    "tcy": ["ವರ್ಗೊ"],
    "te": ["వర్గం"],
    "tet": ["Kategoría", "Kategoria"],
    "tg": ["Гурӯҳ"],
    "th": ["หมวดหมู่"],
    "ti": ["መደብ"],
    "tk": ["Kategoriýa"],
    "tl": ["Kategorya", "Kaurian"],
    "tpi": ["Grup"],
    "tr": ["Kategori", "KAT"],
    "tt": ["Төркем", "Törkem", "Категория"],
    "ty": ["Catégorie"],
    "tyv": ["Аңгылал", "Категория"],
    "udm": ["Категория"],
    "ug": ["تۈر"],
    "uk": ["Категория", "Категорія"],
    "ur": ["زمرہ"],
    "uz": ["Turkum", "Kategoriya"],
    "vec": ["Categoria"],
    "vep": ["Kategorii"],
    "vi": ["Thể_loại", "Thể loại"],
    "vls": ["Categorie"],
    "vo": ["Klad"],
    "wa": ["Categoreye"],
    "war": ["Kaarangay"],
    "wo": ["Wàll", "Catégorie"],
    "wuu": ["分类"],
    "xal": ["Янз", "Әәшл"],
    "xmf": ["კატეგორია"],
    "yi": ["קאטעגאריע", "קאַטעגאָריע"],
    "yo": ["Ẹ̀ka"],
    "za": ["分类"],
    "zea": ["Categorie"],
    "zh": ["分类", "分類", "CAT"],
    "zh-classical": ["分類", "CAT"],
    "zh-min-nan": ["分類", "Lūi-pia̍t"],
    "zh-yue": ["分类", "分類", "类", "類"],
}

In [5]:
import re
import mwparserfromhell

def parse_and_clean_wikitext(wikitext, lang='en'):
    """Strips formatting and unwanted sections from raw page content."""
    wikicode = mwparserfromhell.parse(wikitext)

    # Filters for magic words that are parser instructions -- e.g., __NOTOC__
    re_rm_magic = re.compile("__[A-Z]*__", flags=re.UNICODE)

    # Filters for file/image links.
    media_prefixes = '|'.join(['File', 'Image', 'Media'] + MEDIA_ALIASES.get(lang, []))
    re_rm_wikilink = re.compile(f"^(?:{media_prefixes}):", flags=re.IGNORECASE | re.UNICODE)

    def rm_wikilink(obj):
        return bool(re_rm_wikilink.match(str(obj.title)))

    # Filters for references and tables
    def rm_tag(obj):
        return str(obj.tag).lower() in {"ref", "table", "div", "gallery"}

    # Leave category links in-place but remove the category prefixes
    cat_prefixes = '|'.join(['Category'] + CAT_ALIASES.get(lang, []))
    re_clean_wikilink = re.compile(f"^(?:{cat_prefixes}):", flags=re.IGNORECASE | re.UNICODE)

    def is_category(obj):
        return bool(re_clean_wikilink.match(str(obj.title)))

    def clean_wikilink(obj):
        text = obj.__strip__()
        text = re.sub(re_clean_wikilink, '', text)
        obj.text = text

    def try_replace_obj(obj):
        try:
            clean_wikilink(obj)
        except ValueError:
            # For nested objects, they've sometimes already been removed.
            pass

    def try_remove_obj(obj, section):
        try:
            section.remove(obj)
        except ValueError:
            # For nested objects, they've sometimes already been removed.
            pass

    section_text = []
    # Filter individual sections to clean.
    for section in wikicode.get_sections(flat=True, include_lead=True, include_headings=True):
        for obj in section.ifilter_wikilinks(recursive=True):
            if rm_wikilink(obj):
                try_remove_obj(obj, section)
            elif is_category(obj):
                try_replace_obj(obj)
        for obj in section.ifilter_tags(matches=rm_tag, recursive=True):
            try_remove_obj(obj, section)

        section_text.append(re.sub(re_rm_magic, '', section.strip_code().strip()))
    #return " ".join(section_text)
    text = " ".join(section_text)
    return clean(text)

In [6]:
common_forbidden_patterns =  [
    "<!--.*?-->",  # this would remove any commented-out text, which is rare but sometimes substantial
    "\[\[category:.*?\]\]", # remove categories
    "{{.*?}}", # remove template text
    "&amp;",
    "&lt;",
    "&gt;",
    r"<ref[^<]*<\/ref>",  # remove reference tags
    "<[^>]*>",
    "\|left",
    "\|\d+px",
    r"(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b",
    "\|thumb",
    "\|right",
    "\[\[image:[^\[\]]*",
    "\[\[category:([^|\]]*)[^]]*\]\]",
    "\[\[[a-z\-]*:[^\]]*\]\]",
    "\[",
    "\]",
    "\{[^\}]*\}",
    r"\n",
    " +",
    r"[^a-zA-Z0-9 ]",
    r"\b[a-zA-Z]\b"
]

compiled_patterns = []
for pattern in common_forbidden_patterns:
    compiled_patterns.append(re.compile(pattern))

In [7]:
def clean(text):
    for p in compiled_patterns:       
        plaintext = re.sub(p, ' ', text)
        return ' '.join(plaintext.split())

In [8]:
#pip install mwparserfromhell

In [9]:
plain_pages['cleaned_text'] = plain_pages['plain'].apply(lambda x: parse_and_clean_wikitext(x))

In [10]:
import nltk
import spacy
nlp = spacy.load('en_core_web_sm')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))  
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
ps = PorterStemmer()
tokenizer = RegexpTokenizer(r'\w+')
from string import digits
remove_digits = str.maketrans('', '', digits)
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def preprocess(content):
    if isinstance(content, str):
        processed = content.translate(remove_digits)
    else:
        processed = content
    processed = processed.lower()
    processed = tokenizer.tokenize(processed)
    processed = [lemmatizer.lemmatize(token) for token in processed ]
    processed = [token for token in processed if token not in stop_words]

    return processed

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(tokenizer = preprocess)

In [11]:
from revBERT.keybert import KeyBERT
kw_model = KeyBERT()

In [12]:
def feature_extract(data):
    #keywords = kw_model.extract_keywords(data, top_n = 30, vectorizer = vectorizer)
    try:
        keywords = kw_model.extract_keywords(data, top_n = 20, vectorizer = vectorizer)
        #print(keywords)
        return str(keywords)
    except:
        return []

In [14]:
import warnings
warnings.filterwarnings("ignore")
plain_pages['keywords'] = plain_pages['cleaned_text'].apply(lambda x: feature_extract(x))

In [15]:
plain_pages.rename(columns={'id':'page_id'},inplace=True)
plain_pages.head()

,page_id,title,url,plain,cleaned_text,keywords
0,16897736,Jon Batiste,https://en.wikipedia.org/wiki/Jon_Batiste,"Jonathan Michael Batiste (born November 11, 19...","Jonathan Michael Batiste (born November 11, 19...","[('batiste', 0.3953), ('musician', 0.3645), ('..."
1,2853,Aberdeen Bestiary,https://en.wikipedia.org/wiki/Aberdeen_Bestiary,The Aberdeen Bestiary (Aberdeen University Lib...,The Aberdeen Bestiary (Aberdeen University Lib...,"[('bestiary', 0.4735), ('manuscript', 0.3808),..."
2,882558,Ray Allen,https://en.wikipedia.org/wiki/Ray_Allen,"Walter Ray Allen Jr. (born July 20, 1975) is a...","Walter Ray Allen Jr. (born July 20, 1975) is a...","[('allen', 0.54), ('kyrie', 0.3569), ('nba', 0..."
3,13467,Hamburg,https://en.wikipedia.org/wiki/Hamburg,"Hamburg (, ; ), officially the Free and Hanse...","Hamburg (, ; ), officially the Free and Hansea...","[('hamburg', 0.6202), ('hamburgische', 0.5974)..."
4,853925,Randy Kraft,https://en.wikipedia.org/wiki/Randy_Kraft,"Randy Steven Kraft (born March 19, 1945) is an...","Randy Steven Kraft (born March 19, 1945) is an...","[('homicide', 0.3784), ('murderer', 0.3714), (..."


def only_keys(col):
    l = []
    for i in col.split('),'):
        #print(i.split(',')[0][2:])
        l.append(i.split(',')[0][2:])
    return l

In [16]:
import math
import re
def TI(row):
    d = {}
    #N = len(row)
    #key_lists = row.values
    for keys in row:
        #print(keys)
        key_split = keys.split('),')
        for key in key_split:
            token_val = key.split(',')
            #token = token_val[0].replace("_", " ")
            token = re.sub(r'[^\w\s|\s+|\s+$]','', token_val[0]).strip() #no space
            token = lemmatizer.lemmatize(token)
            val = re.sub(r'[^\-|\.|0-9]','', token_val[1]) #no number
            if token in d.keys():
                d[token]+= float(val)
                #d[token]['TI'] += float(val)
                #d[token]['DF'] += 1
            else: 
                d[token] = float(val)
                #d[token] = {}
                #d[token]['TI'] = float(val)
                #d[token]['DF'] = 1
            #d['token']['TI'] =  round(d['token']['TI'],3)
            #d['token']['DF'] = log(N/d['toke']['DF'])
                
    return d

In [17]:
def measure_TI(df):
    all_df = pd.DataFrame()
    all_df['keys'] = df.groupby('topic_id')['keywords'].apply(list)
#l = sample.groupby('item')['keywords'].apply(lambda x: IDF(x))
    all_df = all_df.reset_index()
    all_df['TI'] = all_df['keys'].apply(lambda x: TI(x))
    return all_df

In [18]:
topic_info = pd.read_csv('data/topic_info_2022.csv')
topic_info

,Unnamed: 0,id,title,page_id
0,0,84,Agriculture,585460
1,1,84,Agriculture,20656228
2,2,84,Agriculture,11107
3,3,84,Agriculture,333606
4,4,84,Agriculture,62696
...,...,...,...,...
995,995,2872,Women's sport,9509330
996,996,2872,Women's sport,31804969
997,997,2872,Women's sport,3517532
998,998,2872,Women's sport,21031813


In [20]:
topic_page = pd.merge(topic_info[['page_id', 'id', 'title']], plain_pages[['page_id', 'title', 'keywords']], on ='page_id', how='inner')
topic_page.rename(columns={'title_x':'topic_title', 'title_y':'page_title', 'id':'topic_id'},inplace=True)
topic_page.head()

,page_id,topic_id,topic_title,page_title,keywords
0,585460,84,Agriculture,Cherimoya,"[('cherimoya', 0.511), ('cherimola', 0.4908), ..."
1,20656228,84,Agriculture,Maize,"[('maize', 0.6339), ('corn', 0.6263), ('cornbr..."
2,11107,84,Agriculture,Food and Agriculture Organization,"[('organization', 0.401), ('organisation', 0.3..."
3,11107,1328,International relations,Food and Agriculture Organization,"[('organization', 0.401), ('organisation', 0.3..."
4,333606,84,Agriculture,List of chicken breeds,"[('poultry', 0.5437), ('chicken', 0.3774), ('p..."


In [21]:
all_df = measure_TI(topic_page)

In [22]:
all_df.head(5)

,topic_id,keys,TI
0,84,"[[('cherimoya', 0.511), ('cherimola', 0.4908),...","{'cherimoya': 0.511, 'cherimola': 0.4908, 'fru..."
1,111,"[[('serpent', 0.5331), ('biblical', 0.4403), (...","{'serpent': 0.8445, 'biblical': 0.4403, 'genes..."
2,187,"[[('library', 0.4832), ('librarian', 0.4761), ...","{'library': 0.7518, 'librarian': 0.803, 'bookb..."
3,265,"[[('astronomer', 0.5958), ('keplerian', 0.5454...","{'astronomer': 2.1547, 'keplerian': 0.5454, 'k..."
4,270,"[[('balance', 0.464), ('nike', 0.3835), ('rebr...","{'balance': 0.464, 'nike': 0.3835, 'rebrand': ..."


import math
def TI_IDF(df, all_tokens):
    TIIDF = {}
    N = len(df['keys'])
    for keys in df['TI_DF']:
        all_tokens.add(keys)
        DF = df['TI_DF'][keys]['DF']
        TI = df['TI_DF'][keys]['TI']
        #IDF = math.log(N/(DF+1))
        #TIIDF[keys] = round(TI * IDF,3)
        TIIDF[keys] = round(TI ,3)
    return TIIDF

all_tokens = set()
all_df['TI_IDF'] = all_df.apply(lambda x: TI_IDF(x, all_tokens), axis=1)
all_df.head()

In [23]:
def top_keywords(row):
    return [k for k, v in sorted(row.items(), key=lambda item: item[1])][::-1]

In [24]:
all_df['top_keys'] = all_df['TI'].apply(lambda x: top_keywords(x))
topic_keys = all_df[['topic_id', 'top_keys']]
topic_keys

,topic_id,top_keys
0,84,"[cultivated, agricultural, maize, corn, fruit,..."
1,111,"[reptile, lizard, salamander, fossil, frog, pr..."
2,187,"[museum, london, baroque, librarian, louvre, a..."
3,265,"[astronomer, astronomy, astrophysicist, mathem..."
4,270,"[athletics, athlete, olympic, competed, athlet..."
...,...,...
95,2741,"[arcade, gaming, nintendo, sega, playstation, ..."
96,2758,"[artwork, art, artistic, painting, painter, sc..."
97,2859,"[tennis, kim, olympic, singer, olympics, athle..."
98,2867,"[novelist, writer, actress, author, biography,..."


In [25]:
df = topic_page.drop_duplicates(subset=['topic_id', 'topic_title'], keep='first')

In [26]:
#topic_page
topic_keys = pd.merge(topic_keys, df[['topic_id', 'topic_title']], how='inner', on='topic_id')
topic_keys

,topic_id,top_keys,topic_title
0,84,"[cultivated, agricultural, maize, corn, fruit,...",Agriculture
1,111,"[reptile, lizard, salamander, fossil, frog, pr...",Amphibians and Reptiles
2,187,"[museum, london, baroque, librarian, louvre, a...",Architecture
3,265,"[astronomer, astronomy, astrophysicist, mathem...",Astronomy
4,270,"[athletics, athlete, olympic, competed, athlet...",Athletics
...,...,...,...
95,2741,"[arcade, gaming, nintendo, sega, playstation, ...",Video games
96,2758,"[artwork, art, artistic, painting, painter, sc...",Visual arts
97,2859,"[tennis, kim, olympic, singer, olympics, athle...",Women
98,2867,"[novelist, writer, actress, author, biography,...",Women writers


In [55]:
#topic_keys['topic_title'] = title #topics['title']

In [17]:
topic_keys.to_parquet('data/top_keys_2022.parquet')

In [15]:
topic_keys.rename(columns={'topic_id':'id', 'top_keys':'keywords', 'topic_title':'title'}, inplace = True)
topic_keys.head()

,id,keywords,title
0,84,"[cultivated, agricultural, maize, corn, fruit,...",Agriculture
1,111,"[reptile, lizard, salamander, fossil, frog, pr...",Amphibians and Reptiles
2,187,"[museum, london, baroque, librarian, louvre, a...",Architecture
3,265,"[astronomer, astronomy, astrophysicist, mathem...",Astronomy
4,270,"[athletics, athlete, olympic, competed, athlet...",Athletics


In [14]:
topic_keys = pd.read_parquet('data/top_keys_2022.parquet')

In [16]:
topic_keys = pd.merge(topic_keys, selected_topics_df[['id','rel_docs']], how = 'left', on = 'id')
topic_keys

,id,keywords,title,rel_docs
0,84,"[cultivated, agricultural, maize, corn, fruit,...",Agriculture,"[572, 627, 678, 903, 1193, 1542, 1634, 3751, 3..."
1,111,"[reptile, lizard, salamander, fossil, frog, pr...",Amphibians and Reptiles,"[621, 809, 1380, 6641, 8311, 8937, 13134, 1446..."
2,187,"[museum, london, baroque, librarian, louvre, a...",Architecture,"[682, 954, 1170, 1315, 1322, 1324, 1325, 1435,..."
3,265,"[astronomer, astronomy, astrophysicist, mathem...",Astronomy,"[39, 308, 580, 664, 736, 748, 791, 798, 799, 1..."
4,270,"[athletics, athlete, olympic, competed, athlet...",Athletics,"[5729, 8490, 9623, 10391, 12231, 13791, 16078,..."
...,...,...,...,...
95,2741,"[arcade, gaming, nintendo, sega, playstation, ...",Video games,"[785, 1203, 1980, 2078, 2116, 2141, 2197, 2215..."
96,2758,"[artwork, art, artistic, painting, painter, sc...",Visual arts,"[593, 752, 864, 1212, 1578, 1642, 1647, 1881, ..."
97,2859,"[tennis, kim, olympic, singer, olympics, athle...",Women,"[890, 974, 984, 2126, 2519, 2529, 2847, 3013, ..."
98,2867,"[novelist, writer, actress, author, biography,...",Women writers,"[339, 984, 1000, 1002, 2029, 2314, 2834, 2847,..."


In [28]:
topic_keys.to_excel("data/top_keys_2022.xlsx", engine='xlsxwriter')

In [29]:
topic_keys

,topic_id,top_keys,topic_title
0,84,"[cultivated, agricultural, maize, corn, fruit,...",Agriculture
1,111,"[reptile, lizard, salamander, fossil, frog, pr...",Amphibians and Reptiles
2,187,"[museum, london, baroque, librarian, louvre, a...",Architecture
3,265,"[astronomer, astronomy, astrophysicist, mathem...",Astronomy
4,270,"[athletics, athlete, olympic, competed, athlet...",Athletics
...,...,...,...
95,2741,"[arcade, gaming, nintendo, sega, playstation, ...",Video games
96,2758,"[artwork, art, artistic, painting, painter, sc...",Visual arts
97,2859,"[tennis, kim, olympic, singer, olympics, athle...",Women
98,2867,"[novelist, writer, actress, author, biography,...",Women writers


In [18]:
eval_topics = pd.read_json('data/trec_2022_eval_reldocs.jsonl', lines = True)
eval_topics

,id,title,rel_docs
0,187,Architecture,"[682, 954, 1170, 1315, 1322, 1324, 1325, 1435,..."
1,270,Athletics,"[5729, 8490, 9623, 10391, 12231, 13791, 16078,..."
2,359,Baseball,"[1135, 1136, 1293, 1893, 2129, 2140, 3797, 380..."
3,365,Basketball,"[3921, 4871, 10871, 14443, 16899, 20132, 20455..."
4,400,Biography/arts and entertainment work group,"[339, 628, 888, 924, 954, 980, 983, 984, 1028,..."
5,404,Biography/sports and games work group,"[595, 890, 1020, 1135, 1293, 1806, 1893, 2575,..."
6,480,COVID-19,"[4191, 6811, 24723, 30919, 33583, 45570, 55214..."
7,517,Canada/WikiProject Political parties and polit...,"[1235, 5981, 12233, 15908, 15912, 15915, 17962..."
8,568,Catholicism,"[1252, 1301, 1370, 1408, 1409, 1513, 1552, 157..."
9,596,Chemistry,"[639, 656, 657, 902, 1014, 1187, 1206, 1207, 1..."


In [19]:
eval_topics = eval_topics['id'].values
eval_topics

array([ 187,  270,  359,  365,  400,  404,  480,  517,  568,  596,  715,
        807,  834,  881,  883,  949,  951,  955,  995, 1018, 1180, 1233,
       1328, 1406, 1417, 1448, 1449, 1479, 1499, 1548, 1558, 1647, 1685,
       1806, 1821, 1877, 1884, 1890, 2000, 2028, 2106, 2153, 2160, 2229,
       2244, 2448, 2483, 2758, 2867, 2872])

In [20]:
eval_topic_keywords = topic_keys[topic_keys['id'].isin(eval_topics)]
eval_topic_keywords.reset_index(drop=True, inplace = True)
eval_topic_keywords

,id,keywords,title,rel_docs
0,187,"[museum, london, baroque, librarian, louvre, a...",Architecture,"[682, 954, 1170, 1315, 1322, 1324, 1325, 1435,..."
1,270,"[athletics, athlete, olympic, competed, athlet...",Athletics,"[5729, 8490, 9623, 10391, 12231, 13791, 16078,..."
2,359,"[stadium, outfielder, hitter, pitcher, mlb, nc...",Baseball,"[1135, 1136, 1293, 1893, 2129, 2140, 3797, 380..."
3,365,"[fiba, nba, basketball, lakers, kobe, knicks, ...",Basketball,"[3921, 4871, 10871, 14443, 16899, 20132, 20455..."
4,400,"[sculpture, architect, aemilius, museum, sculp...",Biography/arts and entertainment work group,"[339, 628, 888, 924, 954, 980, 983, 984, 1028,..."
5,404,"[golfer, golf, pga, competed, nicklaus, athlet...",Biography/sports and games work group,"[595, 890, 1020, 1135, 1293, 1806, 1893, 2575,..."
6,480,"[pandemic, covid, epidemic, coronavirus, outbr...",COVID-19,"[4191, 6811, 24723, 30919, 33583, 45570, 55214..."
7,517,"[trudeau, canadian, minister, quebec, ontario,...",Canada/WikiProject Political parties and polit...,"[1235, 5981, 12233, 15908, 15912, 15915, 17962..."
8,568,"[papacy, jesuit, pope, papal, catholic, saint,...",Catholicism,"[1252, 1301, 1370, 1408, 1409, 1513, 1552, 157..."
9,596,"[chemist, chemistry, molecular, molecule, crys...",Chemistry,"[639, 656, 657, 902, 1014, 1187, 1206, 1207, 1..."


In [21]:
train_topic_keywords = topic_keys[~topic_keys['id'].isin(eval_topics)]
train_topic_keywords.reset_index(drop=True, inplace = True)
train_topic_keywords

,id,keywords,title,rel_docs
0,84,"[cultivated, agricultural, maize, corn, fruit,...",Agriculture,"[572, 627, 678, 903, 1193, 1542, 1634, 3751, 3..."
1,111,"[reptile, lizard, salamander, fossil, frog, pr...",Amphibians and Reptiles,"[621, 809, 1380, 6641, 8311, 8937, 13134, 1446..."
2,265,"[astronomer, astronomy, astrophysicist, mathem...",Astronomy,"[39, 308, 580, 664, 736, 748, 791, 798, 799, 1..."
3,323,"[aviation, afb, usaf, airfield, airport, aeros...",Aviation,"[849, 852, 1293, 1902, 1942, 2039, 2075, 2082,..."
4,396,"[actor, cast, screenwriter, filmmaker, film, a...",Biography/WikiProject Actors and Filmmakers,"[344, 676, 808, 872, 1247, 1806, 1828, 2083, 2..."
5,397,"[singer, musician, songwriter, guitarist, fron...",Biography/WikiProject Musicians,"[700, 859, 864, 875, 880, 905, 1006, 1017, 102..."
6,403,"[philosopher, scientist, mathematician, physic...",Biography/science and academia work group,"[308, 340, 700, 728, 736, 784, 851, 852, 892, ..."
7,409,"[reef, bird, island, plumage, karnataka, benga...",Birds,"[1227, 2995, 3410, 3453, 4193, 4401, 4462, 452..."
8,426,"[qur, novel, trilogy, islamic, mahābhāratas, q...",Books,"[332, 1136, 1756, 2834, 2853, 2956, 3390, 3778..."
9,475,"[rockefeller, financier, businessman, roosevel...",Business,"[1081, 1581, 1806, 1893, 1938, 2391, 2593, 259..."


In [22]:
eval_topic_keywords.to_csv('data/eval_topic_keywords.csv', index = False)
train_topic_keywords.to_csv('data/train_topic_keywords.csv', index = False)

In [41]:
train_topic_keys = pd.read_csv('data/train_topic_keywords.csv')
train_topic_keys.isna().sum()

id          0
keywords    0
title       0
rel_docs    0
dtype: int64

In [42]:
train_topic_keys['id'].nunique()

50

In [43]:
train_topic_keys[train_topic_keys['title']=='Military history/Maritime warfare task force']

,id,keywords,title,rel_docs
37,1656,"['admiral', 'naval', 'captain', 'fleet', 'navy...",Military history/Maritime warfare task force,"[662, 1576, 2219, 2563, 2988, 3147, 3218, 4031..."


In [44]:
train_topic = pd.read_excel('data/train_manual.xlsx')
#train_topic['id'] = train_topic['id'].astype(int)

In [45]:
train_topic['id'].nunique()

46

In [46]:
train_topic_keys_manual = pd.merge(train_topic, train_topic_keys[['id', 'title', 'rel_docs']], on = ['id', 'title'], how = 'left')
train_topic_keys_manual

,id,keywords,title,rel_docs
0,84,"['cultivated', 'agricultural', 'maize', 'corn'...",Agriculture,"[572, 627, 678, 903, 1193, 1542, 1634, 3751, 3..."
1,111,"['reptile', 'lizard', 'salamander', 'fossil', ...",Amphibians and Reptiles,"[621, 809, 1380, 6641, 8311, 8937, 13134, 1446..."
2,265,"['astronomer', 'astronomy', 'astrophysicist', ...",Astronomy,"[39, 308, 580, 664, 736, 748, 791, 798, 799, 1..."
3,323,"['aviation', 'airfield', 'airport', 'aerospace...",Aviation,"[849, 852, 1293, 1902, 1942, 2039, 2075, 2082,..."
4,396,"['actor', 'cast', 'screenwriter', 'filmmaker',...",Biography/WikiProject Actors and Filmmakers,"[344, 676, 808, 872, 1247, 1806, 1828, 2083, 2..."
5,397,"['singer', 'musician', 'songwriter', 'guitaris...",Biography/WikiProject Musicians,"[700, 859, 864, 875, 880, 905, 1006, 1017, 102..."
6,403,"['philosopher', 'scientist', 'mathematician', ...",Biography/science and academia work group,"[308, 340, 700, 728, 736, 784, 851, 852, 892, ..."
7,409,"['bird', 'island', 'plumage', 'falconidae', 'p...",Birds,"[1227, 2995, 3410, 3453, 4193, 4401, 4462, 452..."
8,426,"['novel', 'trilogy', 'paperback', 'miniseries...",Books,"[332, 1136, 1756, 2834, 2853, 2956, 3390, 3778..."
9,475,"[ 'financier', 'businessman', 'roosevelt', 'ri...",Business,"[1081, 1581, 1806, 1893, 1938, 2391, 2593, 259..."


In [47]:
train_topic_keys_manual['id'].nunique()

46

In [49]:
topic_keys = pd.read_excel('data/top_keys_2022 (1).xlsx')
topic_keys['id'] = topic_keys['topic_id'].astype('int')

In [51]:
topic_keys

,Unnamed: 0,topic_id,top_keys,topic_title,URL,Unnamed: 5,id
0,0.0,84.0,"['cultivated', 'agricultural', 'maize', 'corn'...",Agriculture,https://en.wikipedia.org/wiki/Wikipedia:WikiPr...,"""Agriculture"":""https://en.wikipedia.org/wiki/W...",84
1,1.0,111.0,"['reptile', 'lizard', 'salamander', 'fossil', ...",Amphibians and Reptiles,https://en.wikipedia.org/wiki/Wikipedia:WikiPr...,"""Amphibians and Reptiles"":""https://en.wikipedi...",111
2,2.0,187.0,"['museum', 'london', 'baroque', 'librarian', '...",Architecture,https://en.wikipedia.org/wiki/Wikipedia:WikiPr...,"""Architecture"":""https://en.wikipedia.org/wiki/...",187
3,3.0,265.0,"['astronomer', 'astronomy', 'astrophysicist', ...",Astronomy,https://en.wikipedia.org/wiki/Wikipedia:WikiPr...,"""Astronomy"":""https://en.wikipedia.org/wiki/Wik...",265
4,4.0,270.0,"['athletics', 'athlete', 'olympic', 'competed'...",Athletics,https://en.wikipedia.org/wiki/Wikipedia:WikiPr...,"""Athletics"":""https://en.wikipedia.org/wiki/Wik...",270
...,...,...,...,...,...,...,...
95,95.0,2741.0,"['arcade', 'gaming', 'nintendo', 'sega', 'play...",Video games,https://en.wikipedia.org/wiki/Wikipedia:WikiPr...,"""Video games"":""https://en.wikipedia.org/wiki/W...",2741
96,96.0,2758.0,"['artwork', 'art', 'artistic', 'painting', 'pa...",Visual arts,https://en.wikipedia.org/wiki/Wikipedia:WikiPr...,"""Visual arts"":""https://en.wikipedia.org/wiki/W...",2758
97,97.0,2859.0,"['tennis', 'kim', 'olympic', 'singer', 'olympi...",Women,https://en.wikipedia.org/wiki/Wikipedia:WikiPr...,"""Women"":""https://en.wikipedia.org/wiki/Wikiped...",2859
98,98.0,2867.0,"['novelist', 'writer', 'actress', 'author', 'b...",Women writers,https://en.wikipedia.org/wiki/Wikipedia:WikiPr...,"""Women writers"":""https://en.wikipedia.org/wiki...",2867


In [52]:
train_topics_all = pd.merge(train_topic_keys_manual, topic_keys[['id', 'URL']], on = 'id', how='inner')
train_topics_all

,id,keywords,title,rel_docs,URL
0,84,"['cultivated', 'agricultural', 'maize', 'corn'...",Agriculture,"[572, 627, 678, 903, 1193, 1542, 1634, 3751, 3...",https://en.wikipedia.org/wiki/Wikipedia:WikiPr...
1,111,"['reptile', 'lizard', 'salamander', 'fossil', ...",Amphibians and Reptiles,"[621, 809, 1380, 6641, 8311, 8937, 13134, 1446...",https://en.wikipedia.org/wiki/Wikipedia:WikiPr...
2,265,"['astronomer', 'astronomy', 'astrophysicist', ...",Astronomy,"[39, 308, 580, 664, 736, 748, 791, 798, 799, 1...",https://en.wikipedia.org/wiki/Wikipedia:WikiPr...
3,323,"['aviation', 'airfield', 'airport', 'aerospace...",Aviation,"[849, 852, 1293, 1902, 1942, 2039, 2075, 2082,...",https://en.wikipedia.org/wiki/Wikipedia:WikiPr...
4,396,"['actor', 'cast', 'screenwriter', 'filmmaker',...",Biography/WikiProject Actors and Filmmakers,"[344, 676, 808, 872, 1247, 1806, 1828, 2083, 2...",https://en.wikipedia.org/wiki/Wikipedia:WikiPr...
5,397,"['singer', 'musician', 'songwriter', 'guitaris...",Biography/WikiProject Musicians,"[700, 859, 864, 875, 880, 905, 1006, 1017, 102...",https://en.wikipedia.org/wiki/Wikipedia:WikiPr...
6,403,"['philosopher', 'scientist', 'mathematician', ...",Biography/science and academia work group,"[308, 340, 700, 728, 736, 784, 851, 852, 892, ...",https://en.wikipedia.org/wiki/Wikipedia:WikiPr...
7,409,"['bird', 'island', 'plumage', 'falconidae', 'p...",Birds,"[1227, 2995, 3410, 3453, 4193, 4401, 4462, 452...",https://en.wikipedia.org/wiki/Wikipedia:WikiPr...
8,426,"['novel', 'trilogy', 'paperback', 'miniseries...",Books,"[332, 1136, 1756, 2834, 2853, 2956, 3390, 3778...",https://en.wikipedia.org/wiki/Wikipedia:WikiPr...
9,475,"[ 'financier', 'businessman', 'roosevelt', 'ri...",Business,"[1081, 1581, 1806, 1893, 1938, 2391, 2593, 259...",https://en.wikipedia.org/wiki/Wikipedia:WikiPr...


In [62]:
train_topics_all.to_json('data/train_topics_meta.jsonl')

In [57]:
#train_topic_meta = train_topics_all[['id', 'keywords', 'title','URL']]
#train_topic_meta

In [58]:
train_topics_all.to_excel('data/train_topic_meta.xlsx')

In [63]:
pd.read_json('data/train_topics_meta.jsonl')

,id,keywords,title,rel_docs,URL
0,84,"['cultivated', 'agricultural', 'maize', 'corn'...",Agriculture,"[572, 627, 678, 903, 1193, 1542, 1634, 3751, 3...",https://en.wikipedia.org/wiki/Wikipedia:WikiPr...
1,111,"['reptile', 'lizard', 'salamander', 'fossil', ...",Amphibians and Reptiles,"[621, 809, 1380, 6641, 8311, 8937, 13134, 1446...",https://en.wikipedia.org/wiki/Wikipedia:WikiPr...
2,265,"['astronomer', 'astronomy', 'astrophysicist', ...",Astronomy,"[39, 308, 580, 664, 736, 748, 791, 798, 799, 1...",https://en.wikipedia.org/wiki/Wikipedia:WikiPr...
3,323,"['aviation', 'airfield', 'airport', 'aerospace...",Aviation,"[849, 852, 1293, 1902, 1942, 2039, 2075, 2082,...",https://en.wikipedia.org/wiki/Wikipedia:WikiPr...
4,396,"['actor', 'cast', 'screenwriter', 'filmmaker',...",Biography/WikiProject Actors and Filmmakers,"[344, 676, 808, 872, 1247, 1806, 1828, 2083, 2...",https://en.wikipedia.org/wiki/Wikipedia:WikiPr...
5,397,"['singer', 'musician', 'songwriter', 'guitaris...",Biography/WikiProject Musicians,"[700, 859, 864, 875, 880, 905, 1006, 1017, 102...",https://en.wikipedia.org/wiki/Wikipedia:WikiPr...
6,403,"['philosopher', 'scientist', 'mathematician', ...",Biography/science and academia work group,"[308, 340, 700, 728, 736, 784, 851, 852, 892, ...",https://en.wikipedia.org/wiki/Wikipedia:WikiPr...
7,409,"['bird', 'island', 'plumage', 'falconidae', 'p...",Birds,"[1227, 2995, 3410, 3453, 4193, 4401, 4462, 452...",https://en.wikipedia.org/wiki/Wikipedia:WikiPr...
8,426,"['novel', 'trilogy', 'paperback', 'miniseries...",Books,"[332, 1136, 1756, 2834, 2853, 2956, 3390, 3778...",https://en.wikipedia.org/wiki/Wikipedia:WikiPr...
9,475,"[ 'financier', 'businessman', 'roosevelt', 'ri...",Business,"[1081, 1581, 1806, 1893, 1938, 2391, 2593, 259...",https://en.wikipedia.org/wiki/Wikipedia:WikiPr...
